In [1]:
from matplotlib.pyplot import *
import numpy
from IPython.display import display
import csv
import pandas as pd
from praw.models import MoreComments


In [2]:
import praw

reddit = praw.Reddit(client_id='Bmkuwt-Cz8rTgw',
                     client_secret='M2WDVEEaXfQqAwKgibu6YSdh5Ec',
                     password='^*i%bHAi32Yk',
                     user_agent='testscript by /u/rkeb',
                     username='rkeb')

In [3]:
reddit.config.store_json_result = True
subreddit = reddit.subreddit("all")

In [4]:
def get_topics(pandas_series, num_clusters): #tokenize
    from nltk.corpus import stopwords
    stop_words = stopwords.words('english')

    # tokenization
    tokenized_doc = pandas_series.apply(lambda x: x.split())

    # remove stop-words
    tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])

    # de-tokenization
    detokenized_doc = []
    for i in range(len(pandas_series)):
        t = ' '.join(tokenized_doc[i])
        detokenized_doc.append(t)

    sub_comments = detokenized_doc

    #Document-Term Matrix
    from sklearn.feature_extraction.text import TfidfVectorizer

    vectorizer = TfidfVectorizer(stop_words='english', 
                                 #max_features= 500, # keep top 1000 terms 
                                 max_df = 0.5,
                                 use_idf=True,
                                 smooth_idf=True)

    document_term_matrix = vectorizer.fit_transform(pandas_series)
    
    from sklearn.decomposition import TruncatedSVD

    # SVD represent documents and terms in vectors 
    n_topics = num_clusters
    lsa_model = TruncatedSVD(n_components=n_topics, 
                             algorithm='randomized', 
                             n_iter=100, 
                             random_state=122)

    lsa_model.fit(document_term_matrix)
    terms = vectorizer.get_feature_names()
    topics = {}
    for i, comp in enumerate(lsa_model.components_):
        terms_comp = zip(terms, comp)
        sorted_terms = sorted(terms_comp, key= lambda x:x[1], reverse=True)[:7]
        topic_num = "Topic " + str(i) + ": "
        topic_words = ""
        for t in sorted_terms:
            topic_words= topic_words + t[0] + " "
        topics[topic_num] = topic_words
    return topics

In [ ]:
def clean_data(data):
    # removing everything except alphabets`
    data = data.replace("[^a-zA-Z#]", " ")
    # removing short words
    data = data.apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
    # make all text lowercase
    data = data.apply(lambda x: x.lower())  
    return data

## Topic Analysis of ex_addict_bro's comments

In [5]:
import pandas as pd
import datetime as dt

# COMMENTS FOR A USER
sub_comments_dict = { "body":[]}

for comment in reddit.redditor('ex_addict_bro').comments.top('all'):
    sub_comments_dict['body'].append(comment.body)
       
#sub_comments = [sub for sub in sub_comments if not sub.domain.startswith('self.')]

pd.set_option("display.max_colwidth", 200)
topics_data_ex_addict_bro = pd.DataFrame(sub_comments_dict)

# removing everything except alphabets`
topics_data_ex_addict_bro['body'] = topics_data_ex_addict_bro['body'].replace("[^a-zA-Z#]", " ")
# removing short words
topics_data_ex_addict_bro['body'] = topics_data_ex_addict_bro['body'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
# make all text lowercase
topics_data_ex_addict_bro['body'] = topics_data_ex_addict_bro['body'].apply(lambda x: x.lower())    

#display(topics_data_ex_addict_bro)
display(get_topics(topics_data_ex_addict_bro['body'], 8))

{'Topic 0: ': 'don time like post shit girl woman ',
 'Topic 1: ': 'castle mario princess sorry don later thank ',
 'Topic 2: ': 'time post solid probably waste threads banned ',
 'Topic 3: ': 'post told like horse ll solid subreddit ',
 'Topic 4: ': 'time probably maybe like think just people ',
 'Topic 5: ': 'horse okay told thought write friend later ',
 'Topic 6: ': 'good ll fucked start told place women ',
 'Topic 7: ': 'later thank fuck read field report validation '}

## Get Data and save is CSV

#### Get all submission data

In [6]:
# with open('submissions.csv', mode='a') as sub_file:
#    sub_file = csv.writer(sub_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
#    for submission in reddit.subreddit('atheism').new():
#         sub_file.writerow([submission.title, submission.id, submission.subreddit, submission.created_utc, submission.upvote_ratio])
#    for submission in reddit.subreddit('askreddit').new():
#         sub_file.writerow([submission.title, submission.id, submission.subreddit, submission.created_utc, submission.upvote_ratio])
#    for submission in reddit.subreddit('VeganCircleJerk').new():
#         sub_file.writerow([submission.title, submission.id, submission.subreddit, submission.created_utc, submission.upvote_ratio])
#    for submission in reddit.subreddit('vegan').new():
#         sub_file.writerow([submission.title, submission.id, submission.subreddit, submission.created_utc, submission.upvote_ratio])
#    for submission in reddit.subreddit('TodayILearned').new():
#         sub_file.writerow([submission.title, submission.id, submission.subreddit, submission.created_utc, submission.upvote_ratio])
#    for submission in reddit.subreddit('The_Donald').new():
#         sub_file.writerow([submission.title, submission.id, submission.subreddit, submission.created_utc, submission.upvote_ratio])
#    for submission in reddit.subreddit('changemyview').new():
#         sub_file.writerow([submission.title, submission.id, submission.subreddit, submission.created_utc, submission.upvote_ratio])
#    for submission in reddit.subreddit('TheRedPill').new():
#         sub_file.writerow([submission.title, submission.id, submission.subreddit, submission.created_utc, submission.upvote_ratio])
   

## Topic Analysis of titles for Vegan (313k), VeganCircleJerk (23k), AskReddit (221m), TodayILearned (20m), TheRedPill (235k), Atheism (2.4m) for data stored in submissions.csv

In [27]:
comments_dict = { "vegan":[],'vegancirclejerk':[],'AskReddit':[],'TheRedPill':[],'todayilearned':[],
                 'atheism':[],'The_Donald':[],'changemyview':[]}

with open('submissions.csv') as sub_file:
    subreader = csv.reader(sub_file)
    for row in subreader:
        if row[2] == "vegan":
            comments_dict['vegan'].append(row[0])
        elif row[2] == "vegancirclejerk":
            comments_dict['vegancirclejerk'].append(row[0])
        elif row[2] == "AskReddit":
            comments_dict['AskReddit'].append(row[0])
        elif row[2] == "todayilearned":
            comments_dict['todayilearned'].append(row[0])
        elif row[2] == "TheRedPill":
            comments_dict['TheRedPill'].append(row[0])
        elif row[2] == "atheism":
            comments_dict['atheism'].append(row[0])
        elif row[2] == "The_Donald":
            comments_dict['The_Donald'].append(row[0])
        elif row[2] == "changemyview":
            comments_dict['changemyview'].append(row[0])
            
pd.set_option("display.max_colwidth", 200)
topics_data = pd.DataFrame(comments_dict)

display(topics_data)

,vegan,vegancirclejerk,AskReddit,TheRedPill,todayilearned,atheism,The_Donald,changemyview
0,From pita guys in north county San Diego loving all the vegans options 🙌🏻🤤,"""Meat""","Parents of Reddit, what‘s something hilarious your kid did that you couldn’t laugh at?",How to Avoid Validation Traps,"TIL Jim Cramer is barred from trading most stocks. His contract with CNBC only allows him to trade TheStreet, General Electric, and Comcast","The Christians in my life try to sell me their religion, but I don't see it really helping them.","Hi, I'm Mayor Pete. I'm the latest media-created leftist who thinks government is the answer to all your problems. My dad was a Marxist and I'm gay. Ask how conservatives aren't really Christian.",CMV. Men need more experience than women to survive in todays dating world.
1,Ben and Jerry’s launches cookie dough bites,Going in guns blazing today m8,"Fellow tall people of reddit, what's the best tall related insult you've had?",Introducing the Heptameron,"TIL that Pope John Paul ll's funeral resulted in the largest gathering of heads of state, presidents, and religious leaders in all of history. It also inspired the largest christian pilgrimage in ...",Why Religious People are annoying.,Pluto is gone. Now Uranus is next,CMV: PewDiePie is a crypto-fascist.
2,How many vegan pulled-pork sandwiches do you think I can make with this 30 lb jackfruit?,"Enjoy some nice, delicious steak",What old school slang are you happy is out of circulation?,MAWAD,TIL that Microsoft had a book store.,It’s okay to say ‘I don’t know’.,You know you can keep the Democrats fighting longer if you give them small donations because donor counts are determining who is in the debates. I suggest outsiders like Gabbard who tell them off....,CMV: What r/BlackPeopleTwitter did was racist and should have been instantly stopped by mods
3,I hope this guys a vegan 🌱,"Silly vegoon, animals can't be oppressed",Why are pro sports disproportionately represented racially but people complain when other things aren’t?,Picking up strippers... if that’s your bag,TIL an estimated 300 of the worlds largest frog are imported per year into the United States for competitive frog races,"If god doesn’t exist, How were the laws of physics stablished?","""Hispanics Rally to Trump, Boosting His 2020 Chances | RealClearPolitics"" HIGH ENERGY",CMV: The DOJ shouldn't release the entire Mueller Report.
4,"My favorite restaurant in the Yucatán, Nümen. All vegan 🌱",We are about to get a taste of fame and we should all let it go to our protein-deficient heads,What’s the most embarrassing thing you’ve done in public?,Bertrand Russell on women.,TIL That shopping while drunk contributes billions to the US economy,Recovering Muslim here.,"Tucson Sector Border Patrol Arrested 16 Sexual Predators in March - All were encountered within groups of migrants attempting to enter the country illegally. Additionally, the majority of convicti...",CMV: Brands/People who buy fake Instagram followers/likes should be charged with fraud
5,New EarthGrown vegan chocolate cake.. so good 🤤,Vegetarian guilt,What’s the story of the most poorly behaved child you’ve seen in public?,Nobody was hand-held to enlightenment.,"TIL that if you hold in too many farts, the gas can be reabsorbed and then exhaled in your breath.",I listen to Christian and Catholic radio stations while driving every day,dont say that the report is more damaging than barr mad it out to be if,CMV: Nationalism cannot live in harmony with self segregation along lines other than religion.
6,Conflicted Feelings,When will vegetarians acknowledge the health risks associated with their diet?,What is your biggest regret in life so far and (if applicable) what lesson will you pass on?,Sexual Selection and Existential Fear,"TIL Snails have thousands of insanely hard teeth. Sometimes as many as 20,000.","New York yeshiva backtracks on ordaining its first openly gay student, months before his graduation",Half ass news,CMV: The United 

In [29]:

for data in topics_data:
    clean_data(topics_data[data])


In [30]:
print('Topics for The_Donald')
display(get_topics(topics_data['The_Donald'], 8))
print('Topics for changemyview')
display(get_topics(topics_data['changemyview'], 8))
print('Topics for vegan')
display(get_topics(topics_data['vegan'], 8))
print('Topics for vegancirclejerk')
display(get_topics(topics_data['vegancirclejerk'], 8))
print('Topics for AskReddit')
display(get_topics(topics_data['AskReddit'], 8))
print('Topics for TheRedPill')
display(get_topics(topics_data['TheRedPill'], 8))
print('Topics for todayilearned')
display(get_topics(topics_data['todayilearned'], 8))
print('Topics for atheism')
display(get_topics(topics_data['atheism'], 8))

Topics for The_Donald


{'Topic 0: ': 'joe creepy biden media just things hogg ',
 'Topic 1: ': 'trump president democrats military donna 2020 changing ',
 'Topic 2: ': 'time just democrats white welcome nice state ',
 'Topic 3: ': 'time media creepy guys white hard chance ',
 'Topic 4: ': 'just welcome state islam students white anymore ',
 'Topic 5: ': 'pool tim border leftists racist actually college ',
 'Topic 6: ': 'trying make donations counted ensure gets lisa ',
 'Topic 7: ': 'new democratic pot legalizing litmus test ar '}

Topics for changemyview


{'Topic 0: ': 'thing good colorblindness immigration person unmusical business ',
 'Topic 1: ': 'people white racist inherently adopt procreate color ',
 'Topic 2: ': 'waste time voting high mattresses money quality ',
 'Topic 3: ': 'america states united state dinosaur national rex ',
 'Topic 4: ': 'useless worse political research science justice actually ',
 'Topic 5: ': 'right use amendment open change solve 2nd ',
 'Topic 6: ': 'did wrong leafyishere racist blackpeopletwitter instantly mods ',
 'Topic 7: ': 'bad impossible suffering fascist equality reaching basically '}

Topics for vegan


...

{'Topic 0: ': 'pill swallowed finally years seven think red ',
 'Topic 1: ': 'red men women pilled life pill blue ',
 'Topic 2: ': 'women men swallowed don problem existential fear ',
 'Topic 3: ': 'woman love truly having power helped gets ',
 'Topic 4: ': 'men existential fear selection sexual advice born ',
 'Topic 5: ': 'story life helped michael subreddit best having ',
 'Topic 6: ': 'words free don trp matter children raising ',
 'Topic 7: ': 'report field say approached cold mom male '}

Topics for todayilearned


{'Topic 0: ': 'years 000 people used american times movie ',
 'Topic 1: ': 'used germany called city soda australia batmania ',
 'Topic 2: ': 'soda called largest state ok came history ',
 'Topic 3: ': 'movie actor business chunk goonies shown 000 ',
 'Topic 4: ': 'soda years ok came american going airship ',
 'Topic 5: ': 'use soda people actor movie way came ',
 'Topic 6: ': 'children flag court shot actor soldiers right ',
 'Topic 7: ': 'duck dog trevor died actor soda pope '}

Topics for atheism


{'Topic 0: ': 'god plan say christian think religion died ',
 'Topic 1: ': 'religion death just brunei stoning atheism christianity ',
 'Topic 2: ': 'death stoning brunei punishable laws lgbt new ',
 'Topic 3: ': 'christianity arguments just right world trashy gate ',
 'Topic 4: ': 'people religious annoying believe really care non ',
 'Topic 5: ': 'atheism people christian religious parents annoying atheist ',
 'Topic 6: ': 'christianity atheism science right relevant gate trashy ',
 'Topic 7: ': 'atheism just bible bias confirmation people support '}

## Topic Analysis of comments and comment replies for The_Donald, changemyview and vegan, comment data in file submissions_with_comments.csv

#### Get all comments and sub-comments

In [31]:
# with open('submissions_with_sub_comments.csv', mode='a') as sub_file:
#        sub_file = csv.writer(sub_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
#        for submission in reddit.subreddit('vegan').new(limit=500):
#            submission.comments.replace_more(limit=None)
#            for comment in submission.comments.list():
#                sub_file.writerow([submission.title, submission.id, comment.body, comment.parent_id, submission.subreddit, comment.created_utc, submission.upvote_ratio])
#        for submission in reddit.subreddit('The_Donald').new(limit=500):
#            for comment in submission.comments.list():
#                if isinstance(comment, MoreComments):
#                    continue
#                sub_file.writerow([submission.title, submission.id, comment.body, comment.parent_id, submission.subreddit, comment.created_utc, submission.upvote_ratio])
#        for submission in reddit.subreddit('changemyview').new(limit=500):
#            for comment in submission.comments.list():
#                if isinstance(comment, MoreComments):
#                    continue
#                sub_file.writerow([submission.title, submission.id, comment.body, comment.parent_id, submission.subreddit, comment.created_utc, submission.upvote_ratio])


In [84]:
y_the_donald3 = []
y_changemyview3 = []
y_vegan3 = []


with open('submissions_with_sub_comments.csv') as sub_file:
    subreader = csv.reader(sub_file)
    for row in subreader:
        if row[4] == "vegan":
            y_vegan3.append(row[2])
        elif row[4] == "changemyview":
            y_changemyview3.append(row[2])
        elif row[4] == "The_Donald":
            y_the_donald3.append(row[2])
            
y_vegan3 = pd.DataFrame({'vegan':y_vegan3})
for data in y_vegan3:
    clean_data(y_vegan3[data])
display(y_vegan3)

y_changemyview3 = pd.DataFrame({'changemyview':y_changemyview3})
for data in y_changemyview3:
    clean_data(y_changemyview3[data])
display(y_changemyview3)

y_the_donald3 = pd.DataFrame({'The_Donald':y_the_donald3})
for data in y_the_donald3:
    clean_data(y_the_donald3[data])
display(y_the_donald3)




,vegan
0,"We make this **avocado pudding** at home all the time - it's brilliant! It does pudding *way* better than dairy *ever* did.\n\n#Ingredients:\n\n* 1 ripe avocado, peeled and pitted\n* 1⁄4 cup dry b..."
1,I cannot recommend strongly enough my **Vegan-Friendly Mac n’ “Cheese” with “Ham”** recipe; Kraft Dinner has nothing on this dish. Always a crowd pleaser.\n\n#Ingredients:\n\n* 1 recipe of Seasone...
2,"Creating **tofu steaks** is easy, and it’s a flavorful and healthy way to add variety to your meals. These work great on their own, or topped with things like sauerkraut and/or condiments like mus..."
3,Gross I like chikin nuggets better
4,"How much was that, $60 ?"
5,Make corn dogs!!
6,I'd guess none since there isn't any pork...
7,It was $25 and change. At a local Asian grocery store.
8,Pulled jackfruit doesn’t really have a nice ring to it lol
9,"Nice dude. Ive always been to intimidated to buy one. The jackfruit I normally see are around 20-30 bucks, but I don't think they are that big"


,changemyview
0,>Most men aren't attractive to most women. And because women are the choosers the are enamoured with male attention left and right and try to find the best possible partner for herself.\n\nThis is...
1,"I wouldn’t agree that dating is harder for men. I’m a male, and while I’ve had lots of trouble finding a partner, I’ve never thought that that had to do with my gender, maybe it is. I assumed that..."
2,">Women simply hold all the best cards for her not be desperate for nearly any man. They hold the power to sex, they have multiple options, they are less likely than men fall in love, they rarely g..."
3,"Could you further elaborate on what you mean by ""surviving"" in today's dating world?"
4,"> Why are more than half of divorces initiated by women.\n\nThe answer might be in part that women are more likely to initiate a divorce without having found another mate. Whereas, men are more li..."
5,I actually think you have that backwards
6,"Do you have a reason for disagreeing? My doubt about the idea is not because it's unpopular. Quite the opposite, it's too popular: bring to mind the trope of a middle-aged man initiating a divorce..."
7,/u/aliteralSJW (OP) has awarded 1 delta(s) in this post.\n\nAll comments that earned deltas (from OP or other users) are listed [here](/r/DeltaLog/comments/b95u2l/deltas_awarded_in_cmv_pewdiepie_i...
8,Are these your own beliefs or talking points you got elsewhere? I only ask that because Pewdiepies followed page on Twitter only lists Dr Phil and a Korean boy band.\n\nOr do you mean to say his “...
9,"> There are countless threads on 4chan and other alt-right forums claiming him as their own.\n\n\nSo what? That doesn't mean anything. If Antifa starts ""claiming Obama as their own"" that wouldn't ..."


,The_Donald
0,Must be Brennan and Clapper making up stuff again.
1,The democrats can not handle the truth.
2,Clinton Russian collusion.
3,[http://patriotretort.com/just-how-clueless-is-kirsten-gillibrand/](http://patriotretort.com/just-how-clueless-is-kirsten-gillibrand/)
4,👍
5,"Fox also sucks, sub to pewdiepie."
6,Fake news
7,"I opened the Apple News app earlier, and this was one of the first featured stories. The article is opinion attempting to be passed off as fact, which is the liberal media's modus operandi."
8,Papa D suggests Mueller will expose some FBI corruption. Also the only thing they have on Trump are the questions he answered. He doesn't text or write emails
9,Lmao gotta hold on hope to something


In [89]:
print('Topics for The_Donald')
display(get_topics(y_the_donald3['The_Donald'], 8))
print('Topics for vegan')
display(get_topics(y_vegan3["vegan"], 8))
print('Topics for changemyview')
display(get_topics(y_changemyview3['changemyview'], 8))


Topics for The_Donald


{'Topic 0: ': 'just like don people fuck good lol ',
 'Topic 1: ': 'removed automatically bot the_donald compose contact performed ',
 'Topic 2: ': 'kek removed creepy holy reason copypasta ouch ',
 'Topic 3: ': 'automatically bot https the_donald compose contact performed ',
 'Topic 4: ': 'lol good pathetic awesome guy wtf work ',
 'Topic 5: ': 'good time job point work feels idea ',
 'Topic 6: ': 'fuck good job army holy damn point ',
 'Topic 7: ': 'just like joe biden looks did pedo '}

Topics for vegan


{'Topic 0: ': 'vegan like just don meat people eat ',
 'Topic 1: ': 'deleted memory posted comment right slur think ',
 'Topic 2: ': 'removed vegan staple fries non bowls hates ',
 'Topic 3: ': 'vegan https com www veganactivism org options ',
 'Topic 4: ': 'thanks ll thank vegan try good check ',
 'Topic 5: ': 'thank veganactivism https org com bot www ',
 'Topic 6: ': 'burger impossible king lol meat https com ',
 'Topic 7: ': 'thank lol good looks burger vegan like '}

Topics for changemyview


{'Topic 0: ': 'removed changemyview wiki reddit com www https ',
 'Topic 1: ': 'removed deleted notification screencaps manufactures instructive purposeful ',
 'Topic 2: ': 'people don just think like want make ',
 'Topic 3: ': 'deleted mod misgendering probably careful subpoena username ',
 'Topic 4: ': 'appeal message rule http comments comment rules ',
 'Topic 5: ': 'deltalog op comments view change deltas mean ',
 'Topic 6: ': 'people white women men gender sex trans ',
 'Topic 7: ': 'people money white insurance pay government deltalog '}